In [3]:
import mechanicalsoup
import requests
import os
import tika
tika.initVM()
from tika import parser

## Define the functions for searching with duckduckgo

In [ ]:
def search_with_duckduckgo(company):
    browser = connect_to_duckduckgo()
    hits = []
    try:
        browser.select_form('#search_form_homepage')
        query = company + " whitepaper filetype:pdf"
        browser["q"] = query
        browser.submit_selected()
        hits = browser.get_current_page().select('a.result__a')
    finally:
        browser.close()
    return hits

def connect_to_duckduckgo():
    browser = mechanicalsoup.StatefulBrowser(
        user_agent='Mozilla/65',
        raise_on_404=True,
    )
    browser.open("https://duckduckgo.com/")
    return browser

## Load the data

In [ ]:
with open('Misc/ICO_company_names.txt', 'r') as f:
    company_names = [i.strip() for i in f.readlines()]
companies_to_scrape = company_names

## Search with duckduckgo

In [ ]:
c = 0
while companies_to_scrape:
    company = companies_to_scrape[0]
    print("\n" + company)
    companies_to_scrape.pop(0)
    
    hits = search_with_duckduckgo(company)
    if not hits:
        companies_to_scrape.append(company)
        continue
        c += 1
    else:
        c = 0
        for hit in hits:
            href = hit.attrs['href']
            print(href)
            try:
                r = requests.get(href)
                if r.status_code == 200:
                    with open("duckduckgo_results/" + company + ".pdf", 'wb') as f:
                        f.write(r.content)
                        print('success')
                    break
                else:
                    continue
            except: 
                continue
    if c > len(company_names):
        break

## Evaluate Search Result

In [ ]:
scraped = [f[:-4] for f in os.listdir('duckduckgo_results')]
positives = [f for f in company_names if f in scraped]
negatives = [f for f in company_names if f not in scraped]
print('Positives: ', len(positives))
print('Negatives: ', len(negatives))
print(missing)

In [ ]:
true_positives = []
false_positives = []

for company in positives:
    print(company)
    
    raw = parser.from_file('duckduckgo_results/' + company +'.pdf')
    raw_text = raw['content']
    
    try:
        if company.lower() in raw_text.lower():
            true_positives.append(company)
        else:
            false_positives.append(company)
    except:
        false_positives.append(company)

print('True positives: ', len(true_positives))
print('False positives: ', len(false_positives))

In [ ]:
print(false_positives)